# Determine Word to PCA Associations

This notebook is designed to run PCA over the document embeddings and calculate words-pca associations and document centroid-pca associations for each principal component.

In [1]:
from pathlib import Path
import os
import re

from gensim.models import Doc2Vec
import itertools
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import plotnine as p9
from PIL import ImageColor
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
from tqdm import tqdm_notebook
import wordcloud

matplotlib.use("SVG")  # set the backend to SVG

In [2]:
journal_map_df = pd.read_csv(
    "../exploratory_data_analysis/output/biorxiv_article_metadata.tsv", sep="\t"
)
journal_map_df.head()

,author_type,heading,category,document,doi
0,regular article,new results,genetics,440735_v1.xml,10.1101/440735
1,regular article,new results,systems biology,775270_v1.xml,10.1101/775270
2,regular article,new results,genetics,242404_v1.xml,10.1101/242404
3,regular article,new results,neuroscience,872994_v1.xml,10.1101/2019.12.11.872994
4,regular article,new results,developmental biology,080853_v2.xml,10.1101/080853


# Get the Word Vectors

Save the word vectors to disk, so later sections have easy access.

In [3]:
model = Doc2Vec.load(
    "../word_vector_experiment/output/doc2vec_output/biorxiv_300.model"
)

# PCA the Documents

Run PCA over the documents. Generates 50 principal components, but can generate more or less.

In [4]:
n_components = 50
random_state = 100

In [5]:
reducer = PCA(n_components=n_components, random_state=random_state)

embedding = reducer.fit_transform(model.docvecs.vectors_docs)

pca_df = (
    pd.DataFrame(
        embedding, columns=[f"pca{dim}" for dim in range(1, n_components + 1, 1)]
    )
    .assign(document=[f"{str(tag)}.xml" for tag in model.docvecs.doctags])
    .merge(journal_map_df[["category", "document", "doi"]], on="document")
)

In [6]:
(
    pd.DataFrame(
        reducer.components_,
        columns=[f"{dim+1}" for dim in range(reducer.components_.shape[1])],
    ).to_csv(
        "output/word_pca_similarity/pca_components_doc2vec.tsv", sep="\t", index=False
    )
)

# Calculate Word-PCA Cosine Similarity

Once PCA has finished, there are now 50 different principal components. The association between every word and principal component is calculated via [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity) (cosine of the angle between two vectors).

In [7]:
# 1 - cosine distance = cosine similarity
word_pca_similarity = 1 - cdist(model.wv.vectors, reducer.components_, "cosine")

word_pca_similarity.shape

(675764, 50)

In [8]:
word_pca_sim_df = pd.DataFrame(
    word_pca_similarity,
    columns=[f"pca{dim}_cossim" for dim in range(1, n_components + 1, 1)],
).assign(word=model.wv.index2word)

word_pca_sim_df.to_csv(
    f"output/word_pca_similarity/word_pca_cos_sim_{n_components}_pcs_doc2vec.tsv",
    sep="\t",
    index=False,
)

# Remove those pesky citations from the word pca similarity
word_pca_sim_df = word_pca_sim_df.query(
    "~word.str.match(r'^(\(|\[)', na=False)"  # noqa: W605
)
word_pca_sim_df.head()

,pca1_cossim,pca2_cossim,pca3_cossim,pca4_cossim,pca5_cossim,pca6_cossim,pca7_cossim,pca8_cossim,pca9_cossim,pca10_cossim,...,pca42_cossim,pca43_cossim,pca44_cossim,pca45_cossim,pca46_cossim,pca47_cossim,pca48_cossim,pca49_cossim,pca50_cossim,word
0,-0.000729,0.002413,0.001125,0.031829,-0.015342,-0.007332,0.049930,-0.016376,-0.021773,0.003740,...,0.000917,-0.035307,0.093203,0.009636,0.037358,0.002742,0.034404,-0.001231,-0.025573,","
1,-0.032081,-0.057046,-0.055136,0.008052,-0.028604,-0.031474,0.049815,-0.008854,-0.018201,0.040472,...,0.018408,-0.005689,0.046438,0.014136,0.049979,0.017960,0.038572,0.038916,-0.024154,.
2,-0.032362,-0.013502,-0.005475,-0.036873,-0.006743,0.015549,0.005882,-0.006486,0.009503,-0.017492,...,-0.007684,-0.012067,-0.031199,-0.018405,-0.064531,-0.047717,-0.005243,-0.005004,0.010247,)
4,-0.065627,-0.100649,-0.094230,-0.066972,-0.042747,-0.021028,0.002573,0.024973,0.007476,-0.006744,...,-0.001724,0.022482,0.042350,-0.032296,0.020789,-0.007700,0.048178,-0.045664,0.014878,-
5,0.024530,0.000057,0.013701,0.027553,-0.034045,0.012050,0.016951,-0.004791,-0.017976,-0.019640,...,0.011876,0.012081,0.014475,0.001788,0.003556,0.063859,-0.006535,0.011429,-0.030440,-pron-


# Generate Word Clouds for the PC dimensions

Given word to principal component association, next step is to generate word clouds for each principal component. The word clouds have orange representing words that are most similar to the principal component and blue as words most dissimilar to the principal component.

In [9]:
class PolarityColorFunc:
    def __init__(
        self,
        word_class_map,
        positive_key="positive",
        positive="#ef8a62",
        negative_key="negative",
        negative="#67a9cf",
        default="#f7f7f7",
    ):
        self.positive_class = positive
        self.negative_class = negative
        self.positive_key = positive_key
        self.negative_key = negative_key
        self.default_class = default

        self.words_to_color_map = word_class_map

    def get_color_mapper(self, word):
        if word in self.words_to_color_map[self.positive_key]:
            return ImageColor.getrgb(self.positive_class)
        # wordcloud.get_single_color_func(self.positive_class)
        elif word in self.words_to_color_map[self.negative_key]:
            return ImageColor.getrgb(self.negative_class)
        else:
            return ImageColor.getrgb(self.default_class)

    def __call__(self, word, **kwargs):
        return self.get_color_mapper(word)

In [10]:
pca_dimensions = [f"pca{dim}_cossim" for dim in range(1, n_components + 1, 1)]
for pc, component in tqdm_notebook(enumerate(pca_dimensions, start=1)):
    word_class_map = {}

    word_class_map["negative"] = (
        word_pca_sim_df.sort_values(component, ascending=True)
        .head(100)
        .assign(**{component: lambda x: x[component].abs().values.tolist()})
        .assign(**{component: lambda x: x[component] / x[component].max()})[
            ["word", component]
        ]
        .to_dict(orient="records")
    )

    word_class_map["positive"] = (
        word_pca_sim_df.sort_values(component, ascending=False)
        .assign(**{component: lambda x: x[component] / x[component].max()})
        .head(100)[["word", component]]
        .to_dict(orient="records")
    )

    polarity_color_map = PolarityColorFunc(
        {
            word_class: set(map(lambda x: x["word"], word_class_map[word_class]))
            for word_class in word_class_map
        }
    )

    pc = f"{pc}" if pc > 9 else f"0{pc}"

    polarity_cloud = (
        wordcloud.WordCloud(
            background_color="white", width=1024, height=768, collocations=False
        )
        .generate_from_frequencies(
            {
                record["word"]: record[component]
                for word_class in word_class_map
                for record in word_class_map[word_class]
            }
        )
        .recolor(color_func=polarity_color_map)
        .to_file(
            f"output/word_pca_similarity/figure_pieces/pca_{pc}_cossim_word_cloud_doc2vec.png"
        )
    )